In [1]:
import pandas as pd, os, re, numpy as np
%matplotlib inline
from zero_one_based_conversion import convert

In [2]:
samples = pd.read_pickle('../data/full_sample_df.pkl')

In [3]:
#keep tumor normal pairs
# samples = samples[samples.tumor_normal_pair]

In [4]:
project_samples = samples.groupby('project')

In [5]:
for name, group in project_samples:
    print(name, '\n')
    print(group.groupby('tissue_type').size(), '\n')

GST1 

tissue_type
normal    2
tumor     3
dtype: int64 

GTB11 

tissue_type
normal    98
tumor     98
dtype: int64 

GTB19 

tissue_type
normal    1
tumor     2
dtype: int64 

GTB2 

tissue_type
normal    1
tumor     2
dtype: int64 

allegra/AML_Decitabine 

tissue_type
normal     39
tumor     140
dtype: int64 

allegra/AML_Post-Transplant_Relapse 

tissue_type
met         3
normal     17
relapse    16
tumor      16
dtype: int64 

allegra/AML_Quads 

tissue_type
normal    21
tumor     63
dtype: int64 

gue_su/06_MDS-DECITABINE_06-28-manual_review1_4 

tissue_type
normal    20
tumor     20
dtype: int64 

gue_su/SMR_06-09-rAML-review1 

tissue_type
normal    55
tumor     55
dtype: int64 

lee/ALL1 

tissue_type
normal     1
relapse    2
tumor      2
dtype: int64 

lee/AML31 

tissue_type
normal     4
relapse    2
tumor      2
dtype: int64 

lee/BKM120 

tissue_type
normal    17
tumor     17
dtype: int64 

lee/CRC1 

tissue_type
met       4
normal    1
tumor     1
dtype: int64 

lee/DLB

In [6]:
s = samples[['bed_file_path', 'model_id', 'model_type', 'build_id',
         'subject_name', 'individual_name', 'tissue_type', 
         'project', 'tumor_normal_pair', 'has_met', 'has_pre_malignant', 
         'has_relapse', 'sequencing_context', 'species_name', 
         'ref-seq-build']]

In [7]:
review_df = pd.read_pickle('../data/review_df.pkl')
review_df.project = review_df.project.str.strip()

In [8]:
#filter to only tumor normal pairs
review_df = review_df[review_df.individual_name.isin(list(samples.individual_name.unique()))]
review_df.drop_duplicates(inplace=True)

In [9]:
def convert_one_based(row):
    return convert.coordinate_system('\t'.join(map(str,row.values)), 'to_one_based').split('\t')

In [10]:
review_df_one_based = review_df.apply(convert_one_based, axis=1)

Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 11:51579262-51579264->T, the reference and variant fields indicate an insertion variant, however the coordinates (51579262 and 51579264) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 1:36728386-36728388->T, the reference and variant fields indicate an insertion variant, however the coordinates (36728386 and 36728388) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 1:34510640-34510642->AT, the reference and variant fields indicate an insertion variant, however the coordinates (34510640 and 34510642) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 11:51579262-51579264->T, the reference and variant fields indicate an insertion variant, however the coordinates (51579262 and

	Message: For the coordinate 17:7579642-7579659CCCCAGCCCTCCAGGT>0, the reference and variant fields indicate an deletion or substitution variant, however the coordinates (7579642 and 7579659) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 19:35837536-35837538T>0, the reference and variant fields indicate an deletion or substitution variant, however the coordinates (35837536 and 35837538) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 2:102785155-102785157G>0, the reference and variant fields indicate an deletion or substitution variant, however the coordinates (102785155 and 102785157) are not valid for these mutation types.
Coordinate Error in line 1 or input. Coordinate skipped.
	Message: For the coordinate 2:189904250-189904252C>0, the reference and variant fields indicate an deletion or substitution variant, however the coordi

In [11]:
len(review_df)

45870

In [12]:
review_df_one_based = review_df_one_based.replace('',np.nan).dropna(how='all')
len(review_df_one_based)

45618

In [13]:
review_df_one_based.project = review_df_one_based.project.str.strip()

In [14]:
data = pd.merge(s, review_df_one_based, on=['individual_name','project'])

In [15]:
data.project.unique()

array(['gue_su/SMR_06-09-rAML-review1',
       'gue_su/06_MDS-DECITABINE_06-28-manual_review1_4', 'zach/SCLC',
       'tli/gbm', 'tli/rohan', 'tli/taml', 'lee/ALL1', 'lee/AML31',
       'lee/CRC1', 'lee/DLBCL', 'lee/LGG1', 'lee/MPNST',
       'allegra/AML_Decitabine', 'allegra/AML_Post-Transplant_Relapse',
       'allegra/AML_Quads', 'GTB2', 'GTB19', 'traztuzamab', 'GTB11',
       'lymphoma', 'GST1', 'lee/BKM120'], dtype=object)

In [16]:
disease_lookup = {'gue_su/SMR_06-09-rAML-review1' : 'AML',
       'gue_su/06_MDS-DECITABINE_06-28-manual_review1_4': 'AML', 'zach/SCLC': 'SCLC',
       'tli/gbm': 'glioblastoma', 'tli/rohan': 'breast', 'lee/ALL1': 'AML', 'lee/AML31': 'AML', 'lee/CRC1': 'colorectal',
       'lee/DLBCL': 'lymphoma', 'lee/LGG1': 'glioblastoma', 'lee/MPNST': 'MPNST', 'allegra/AML_Decitabine': 'AML',
       'allegra/AML_Post-Transplant_Relapse': 'AML', 'allegra/AML_Quads': 'AML', 'GTB2': 'melanoma',
       'GTB19': 'AML', 'traztuzamab': 'breast', 'GTB11': 'AML', 'lymphoma': 'lymphoma', 'GST1':'GST', 'lee/BKM120':'breast'}

In [17]:
data['disease']=data.project.map(disease_lookup)

In [18]:
reviewer_lookup = {'gue_su/SMR_06-09-rAML-review1' : 'Heather',
       'gue_su/06_MDS-DECITABINE_06-28-manual_review1_4': 'Heather', 'zach/SCLC': 'Lee',
       'tli/gbm': 'Heather', 'tli/rohan': 'Heather', 'lee/ALL1': 'Lee', 'lee/AML31': 'Lee', 'lee/CRC1': 'Lee',
       'lee/DLBCL': 'Lee', 'lee/LGG1': 'Lee', 'lee/MPNST': 'Lee', 'allegra/AML_Decitabine': 'Heather',
       'allegra/AML_Post-Transplant_Relapse': 'Heather', 'allegra/AML_Quads': 'Heather', 'GTB2': 'Avi',
       'GTB19': 'Avi', 'traztuzamab': 'Lee', 'GTB11': 'Lee', 'lymphoma': 'Heather', 'GST1':'Nick', 'lee/BKM120':'Lee'}

In [19]:
data['reviewer'] = data.project.map(reviewer_lookup)

In [20]:
data.groupby('reviewer').size()

reviewer
Avi          1119
Heather     41981
Lee        127169
Nick          505
dtype: int64

In [21]:
data.to_pickle('../data/full_df.pkl')